# CSV 데이터 로드

파일에서 `tf.data.Dataset`로 CSV 데이터를 로드하는 방법에 대해 학습해봅시다.

이번 튜토리얼에서 사용된 데이터는 타이타닉 승객 목록을 가져온 것입니다. 이 모델은 연령, 성별, 티켓 등급 및 단독 여행 여부와 같은 특성을 기반으로 승객의 생존 가능성을 예측합니다.

In [1]:
import functools

import numpy as np
import tensorflow as tf

TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)
print(train_file_path)
print(test_file_path)

16384/13049 [=====================================] - 0s 0us/step
/root/.keras/datasets/train.csv
/root/.keras/datasets/eval.csv


In [2]:
# 넘파이 값을 더 쉽게 읽을 수 있도록 만듭니다.
np.set_printoptions(precision=3, suppress=True)

---
## 1. 데이터 로드하기

In [3]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

`tf.data.experimental.make_csv_dataset` 함수를 사용해서 파일에서 CSV 데이터를 읽고 데이터세트를 작성해 보겠습니다.

[tf.data.experimental.make_csv_dataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset)

In [4]:
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=5,
        label_name=LABEL_COLUMN,  # 레이블 열로 인식할 열
        na_value='?',  # '?'도 빈 칸으로 간주하도록 한다.
        num_epochs=1,  # 1번만 읽는다.
        ignore_errors=True,  # 빈 행이라든지 읽을 수 없는 데이터를 마주하면 건너뛴다.
        **kwargs)  # 다른 인자를 받아들일 칸.
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [5]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print("{:20s}: {}".format(key, value.numpy()))

In [6]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'female']
age                 : [28. 29. 35. 16. 58.]
n_siblings_spouses  : [0 0 0 0 0]
parch               : [0 0 0 0 1]
fare                : [  7.896 211.337   7.896  10.5   153.462]
class               : [b'Third' b'First' b'Third' b'Second' b'First']
deck                : [b'unknown' b'B' b'unknown' b'unknown' b'C']
embark_town         : [b'Southampton' b'Southampton' b'Cherbourg' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'y' b'y' b'n']


현재 csv 파일의 최상단(첫째 줄)에 열 이름이 적혀있습니다. 그래서 데이터세트를 생성할 때 열 이름을 자동으로 선택해서 key값으로 갖습니다.

파일의 첫 번째 줄에 열 이름이 포함되어 있지 않은 경우, 열 이름을 직접 만들어서(CSV_COLUMNS) `tf.data.experimental.make_csv_dataset` 함수의 `**kwargs`로 전달합니다. 인수는 `column_names`입니다.

In [7]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'male' b'male' b'male' b'male']
age                 : [26. 30. 28. 28. 23.]
n_siblings_spouses  : [0 1 0 0 0]
parch               : [0 0 0 0 0]
fare                : [ 7.887 24.     7.225  7.225  7.896]
class               : [b'Third' b'Second' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Cherbourg' b'Cherbourg' b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'y']


데이터세트에서 일부 열을 생략해야 하는 경우 사용하려는 열의 목록(SELECT_COLUMNS)만 작성한 후에 `tf.data.experimental.make_csv_dataset` 함수의 `**kwarg`에 `select_columns` 인수로 전달합니다.

(레이블로 인식할 열은 반드시 포함되어야 합니다)

In [8]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [31. 18. 46. 26.  4.]
n_siblings_spouses  : [1 0 0 2 1]
class               : [b'Third' b'Third' b'First' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'B' b'unknown' b'G']
alone               : [b'n' b'y' b'y' b'n' b'n']


---
## 2. 데이터 전처리

CSV 파일은 다양한 데이터 유형을 포함할 수 있습니다. 일반적으로 데이터를 모델에 공급하기 전에 고정 길이 벡터로 변환합니다.

### 연속 데이터

데이터가 이미 적절한 숫자 형식인 경우, 데이터를 모델로 전달하기 전에 벡터로 묶을 수 있습니다.

In [9]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path,
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [59.  55.5 53.  29.  39. ]
n_siblings_spouses  : [0. 0. 2. 0. 0.]
parch               : [0. 0. 0. 0. 0.]
fare                : [ 7.25   8.05  51.479  9.5   13.   ]


In [10]:
example_batch, labels_batch = next(iter(temp_dataset))
print(example_batch)
print(labels_batch)

OrderedDict([('age', <tf.Tensor: shape=(5,), dtype=float32, numpy=array([16., 29., 31., 28., 30.], dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 8., 0.], dtype=float32)>), ('parch', <tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 0., 0., 2., 0.], dtype=float32)>), ('fare', <tf.Tensor: shape=(5,), dtype=float32, numpy=array([ 8.05 ,  7.046, 52.   , 69.55 ,  7.229], dtype=float32)>)])
tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int32)


다음은 모든 열을 묶는 간단한 함수입니다.

In [11]:
def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label

In [12]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
    print(features.numpy())
    print()
    print(labels.numpy())

[[24.     2.     3.    18.75 ]
 [28.     0.     0.     7.733]
 [28.     0.     0.    56.496]
 [ 1.     1.     2.    20.575]
 [38.     1.     5.    31.388]]

[1 1 1 1 1]


`[age, n_siblings_spouses, parch, fare]` 순으로 묶여 있는 것을 확인할 수 있습니다.

데이터세트가 숫자로만 되어 있지 않고 문자와 혼합이 되어 있다면 어떻게 해야 할까요?

In [13]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'female' b'male']
age                 : [21. 32.  2. 24. 28.]
n_siblings_spouses  : [0 0 3 0 0]
parch               : [0 0 1 0 0]
fare                : [73.5    7.925 21.075 13.    47.1  ]
class               : [b'Second' b'Third' b'Third' b'Second' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'F' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'y' b'n' b'y' b'y']


In [14]:
example_batch, labels_batch = next(iter(temp_dataset))

숫자 특성 목록을 선택하고 단일 열로 묶지 말고 일반적인 전처리를 정의합니다.

In [15]:
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names = names

    def __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]  # 숫자 열들을 기존 데이터세트에서 지운다.
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]  # 지운 숫자 열들의 값들을 실수로
        numeric_features = tf.stack(numeric_features, axis=-1)  # 숫자 데이터들을 묶는다.
        features['numeric'] = numeric_features  # 'numeric'이라는 열을 만들고 숫자 데이터들을 넣는다.

        return features, labels

In [16]:
NUMERIC_FEATURES = ['age', 'n_siblings_spouses', 'parch', 'fare']

packed_train_data = raw_train_data.map(PackNumericFeatures(NUMERIC_FEATURES))
packed_test_data = raw_test_data.map(PackNumericFeatures(NUMERIC_FEATURES))

In [17]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'female' b'female' b'male']
class               : [b'Third' b'Third' b'Third' b'Second' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'y' b'y' b'n' b'n']
numeric             : [[28.     0.     0.     7.896]
 [28.     0.     0.     8.05 ]
 [14.     0.     0.     7.854]
 [22.     1.     1.    29.   ]
 [16.     1.     3.    34.375]]


숫자 데이터들을 따로 빼내서 'numeric'이라는 열로 묶었습니다.

In [18]:
example_batch, labels_batch = next(iter(packed_train_data))

### 데이터 정규화

연속 데이터는 항상 정규화 되어야 합니다.

In [19]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [20]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [21]:
def normalize_numeric_data(data, mean, std):
    return (data-mean)/std

이제 숫자 열을 만듭니다. `tf.feature_columns.numeric_column` API는 normalizer_fn 인수를 받아들입니다.

`functools.partial`를 사용하여 `MEAN`과 `STD`를 노멀라이저 fn에 바인딩합니다.

In [22]:
# 노멀라이저 생성
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)
# 데이터세트에 numeric 열의 값을 정규화
numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]  # 이후 layers.DenseFeatures()에 넣어주기 위해 배열로 만듦.
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f18102d75f0>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

모델을 훈련할 때 이 특성 열을 포함하여 이 숫자 데이터 블록을 선택하고 중앙에 배치합니다.

In [23]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[28.   ,  0.   ,  0.   ,  7.787],
       [28.   ,  0.   ,  0.   ,  7.896],
       [61.   ,  0.   ,  0.   ,  6.238],
       [31.   ,  1.   ,  0.   , 18.   ],
       [28.   ,  0.   ,  0.   , 13.   ]], dtype=float32)>

In [24]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.13 , -0.474, -0.479, -0.487],
       [-0.13 , -0.474, -0.479, -0.485],
       [ 2.507, -0.474, -0.479, -0.516],
       [ 0.109,  0.395, -0.479, -0.3  ],
       [-0.13 , -0.474, -0.479, -0.392]], dtype=float32)

### 범주형 데이터

[tf.feature_column.indicator_column](https://www.tensorflow.org/api_docs/python/tf/feature_column/indicator_column)는 문자열이 언급된 개수만큼 멀티-핫 인코딩으로 나타납니다.

In [25]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [26]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab
    )
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [27]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [28]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]


### 결합된 전처리 레이어

두 개의 연속형 열과 범주형 열을 결합시켜 `tf.keras.layers.DenseFeatures`에 전달합니다.

In [29]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [30]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     1.    -0.13  -0.474
 -0.479 -0.487  0.     1.   ]


## 3. 모델 빌드하기

`preprocessing_layer`를 사용하여 모델을 빌드합니다.

In [31]:
model = tf.keras.Sequential([
                             preprocessing_layer,
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dense(1)
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='Adam',
    metrics=['accuracy']
)

## 4. 훈련, 평가, 예측하기

모델을 인스턴스화하고 훈련할 수 있습니다.

In [32]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [33]:
model.fit(train_data, epochs=20)

Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
126/126 [==============================] - 5s 7ms/step - loss: 0.5100 - accuracy: 0.7432
Epoch 2/20
126/126 [==============================] - 1s 7ms/step - loss: 0.4255 - accuracy: 0.8102
Epoch 3/20
126/126 [==============================] - 1s 6ms/step - loss: 0.4027 - accuracy: 0.8293
Epoch 4/20
126/126 [==============================] - 1s 6ms/step - loss: 0.3959 - accuracy: 0.8246
Epoch 5/20
126/126 [==============================] - 1s 6ms/step - loss: 0.3814 - accuracy: 0.8357
Epoch 6/20
126/126 [==============================] - 1s 6ms/step - loss: 0.3698 - accuracy: 0.8469
Epoch 7/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3665 - accuracy: 0.8325
Epoch 8/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3601 - accuracy: 0.8533
Epoch 9/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3539 - accurac

테스트 데이터를 사용해 정확성을 평가해봅시다.

In [34]:
test_loss, test_accuracy = model.evaluate(test_data)
print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Consider rewriting this model with the Functional API.
53/53 [==============================] - 1s 5ms/step - loss: 0.4704 - accuracy: 0.8561


Test Loss 0.4703662097454071, Test Accuracy 0.8560606241226196


모델 예측을 통해 테스트 데이터의 레이블을 예측해볼 수 있습니다.

In [38]:
predictions = model.predict(test_data)

for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
    prediction = tf.sigmoid(prediction).numpy()
    print(prediction)
    print('Predicted survival: {:.2%}'.format(prediction[0]),
          " | Actual outcome: ",
          ("SURVIVED" if bool(survived) else "DIED"))

[0.916]
Predicted survival: 91.57%  | Actual outcome:  DIED
[0.108]
Predicted survival: 10.79%  | Actual outcome:  DIED
[0.388]
Predicted survival: 38.81%  | Actual outcome:  SURVIVED
[0.988]
Predicted survival: 98.80%  | Actual outcome:  SURVIVED
[0.875]
Predicted survival: 87.52%  | Actual outcome:  SURVIVED
